In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.models import resnet34, EfficientNet_B1_Weights
from torchvision.datasets import CIFAR10
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
from torchmetrics.regression import KendallRankCorrCoef,PearsonCorrCoef
import os
import time
import numpy as np
import pandas as pd

from PIL import Image

from sklearn import preprocessing
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
from torchvision.transforms import AutoAugmentPolicy, AutoAugment, RandomPerspective, RandomAffine, ElasticTransform, Grayscale
from copy import copy

import urllib
import cv2

torch.cuda.is_available()
torch.cuda.get_device_name(0)

In [ ]:
### Model based on Z. Zou, X. Zhang, H. Liu, Z. Li, A. Hussain, i J. Li, „A novel multimodal fusion network based on a joint coding model for lane line segmentation”. http://arxiv.org/abs/2103.11114

In [ ]:
data_root = "./dataStructure/training"
validation_root="./dataStructure/validation"
txt_file_train = "label_2"

thing_classes = ['Pedestrian']
train_img_count = 5

In [ ]:
def get_pedestrian_label(data_root, txt_folder):
    annotations = []
    labels_folder =os.path.join(data_root, txt_folder)
    for file in os.listdir(labels_folder):
        file_id = file[:-4]
        file_path = os.path.join(labels_folder,file)
        with open(file_path, 'r') as opened_file:
            bb_list = []
            for line in opened_file:
              coordinates = line.split()
              if coordinates == []:
                continue
              class_name, _, _,_, xmin, ymin, xmax, ymax, _, _ ,_ , _, _, _, _= map(str, coordinates)
              if class_name in thing_classes:
                xmin = int(float(xmin))
                ymin = int(float(ymin))
                xmax = int(float(xmax))
                ymax = int(float(ymax))
                bb_list.append((xmin,ymin,xmax,ymax))
        annotations.append((file_id,bb_list))

    return annotations

In [ ]:
train_data_dict = get_pedestrian_label(data_root, txt_file_train)
import random

for d in random.sample(train_data_dict, 1):
    image_path = os.path.join(data_root, "image_2", d[0] + ".png")
    img = cv2.imread(image_path)
    h1, w1, _ = img.shape
    for bb in d[1]:
      x1 = bb[0]
      x2 = bb[2]
      y1= bb[1]
      y2 = bb[3]
      cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  

    plt.figure(figsize = (12, 12))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

for d in random.sample(train_data_dict, 1):
    image_path = os.path.join(data_root, "densec", d[0] + ".png")
    imgdense = cv2.imread(image_path)
    imgdense = cv2.resize(imgdense, (w1, h1))
    for bb in d[1]:
        x1, y1, x2, y2 = bb
        cv2.rectangle(imgdense, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    plt.figure(figsize = (12, 12))
    plt.imshow(imgdense)
    plt.show()


In [ ]:
def writeBboxColor(imagename,train_data_dict):
  for label in train_data_dict:
    if label[0]==imagename:
      image_path = os.path.join(data_root, "image_2", imagename + ".png")
      img = Image.open(image_path)
      return img

In [ ]:
def writeBboxDense(imagename,train_data_dict):
  for label in train_data_dict:
    if label[0]==imagename:
      image_path = os.path.join(data_root, "densec", imagename + ".png")
      img = Image.open(image_path)
      return img

In [ ]:
class kitti_loader:
    def __init__(self, directory,transform):
            txt_file_train = "label_2"
            self.root_dir = directory
            self.transform = transform
            self.train_data_dict = get_pedestrian_label(data_root, txt_file_train) 
    def __len__(self):
            return len(os.listdir(self.root_dir+"/calib"))

    def __getitem__(self, idx):
        formatedidx=f"{idx:06}"
        if torch.is_tensor(formatedidx):
            formatedidx = idx.tolist()
        image = writeBboxColor(formatedidx,self.train_data_dict)
        dense = writeBboxDense(formatedidx,self.train_data_dict)
        if self.transform:
            image = self.transform(image)
            dense = self.transform(dense)
        labelList=[]
        for label in train_data_dict:
                if label[0]==formatedidx:
                    if label[1]!=[]:
                        labelList.append(label[1][0])
                    else:
                        labelList.append((0,0,0,0))
        labelList=torch.tensor(labelList)
        return image,dense,labelList

In [ ]:
logdir = "logs"
experiment = 'middle'

# Directory where logs will be saved. 
log_dir = os.path.join(logdir, experiment)

# initiate tensorboard summary writer
tb_writer = SummaryWriter(
    log_dir = log_dir,
    comment = "MiddleFusion"
)
train_accuracy_tag = 'accuracy/train'
validation_accuracy_tag = 'accuracy/validation'
train_loss_tag = 'loss/train'
validation_loss_tag = 'loss/validation'
training_ROC_tag='ROC/train'
validation_ROC_tag='ROC/validation'
training_pearson_tag='pearson/training'
validation_pearson_tag='pearson/validation'
training_kendal_tag='kendal/training'
validation_kendal_tag='kendal/validation'

In [ ]:
def save_model(model, device,epoch, model_dir='models', ):
    model_file_name=f'earlyfusion{epoch}.pth'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)


    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    # transfer the model to gpu.
    if device == 'cuda':
        model.to('cuda')

    return

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Block A pink
        self.A_block_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.A_block_bn = nn.BatchNorm2d(64)
        self.A_block_relu = nn.ReLU()

        # Block B pink
        self.B_block_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.B_block_bn = nn.BatchNorm2d(64)
        self.B_block_relu = nn.ReLU()

        # Block C red
        self.resnet34 = models.resnet34(pretrained=False)
        self.C_resnet34= nn.Sequential(*list(self.resnet34.children())[:-2])
        self.C_resnet34[0] = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3)
        self.C_resnet34 = nn.Sequential(
            self.C_resnet34,
            nn.Conv2d(512, 64, kernel_size=7, stride=2, padding=3)
        )

        # Block D red
        self.D_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.D_resnet34[0] = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3)
        self.D_resnet34 = nn.Sequential(
            self.D_resnet34,
            nn.Conv2d(512, 64, kernel_size=7, stride=2, padding=3)
        )

        #block E red
        self.E_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.E_resnet34[0] = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3)
        self.E_resnet34 = nn.Sequential(
            self.E_resnet34,
            nn.Conv2d(512, 128, kernel_size=7, stride=2, padding=3)
        )

        #block F red
        self.F_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.F_resnet34[0] = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3)
        self.F_resnet34 = nn.Sequential(
            self.F_resnet34,
            nn.Conv2d(512, 128, kernel_size=7, stride=2, padding=3)
        )

        #block G red
        self.G1_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.G1_resnet34[0] = nn.Conv2d(256, 64, kernel_size=7, stride=2, padding=3)
        self.G1_resnet34 = nn.Sequential(
            self.G1_resnet34,
            nn.Conv2d(512, 64, kernel_size=7, stride=2, padding=3)
        )
        self.G2_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.G2_resnet34[0] = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3)
        self.G2_resnet34 = nn.Sequential(
            self.G2_resnet34,
            nn.Conv2d(512, 256, kernel_size=7, stride=2, padding=3)
        )

        #block H pink
        self.H_block_conv = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.H_block_bn = nn.BatchNorm2d(256)
        self.H_block_relu = nn.ReLU()

        #block I pinkpink
        self.I1_block_conv = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.I1_block_bn = nn.BatchNorm2d(256)
        self.I1_block_relu = nn.ReLU()
        self.I2_block_conv = nn.Conv2d(in_channels=512, out_channels=384, kernel_size=3, padding=1)
        self.I2_block_bn = nn.BatchNorm2d(384)
        self.I2_block_relu = nn.ReLU()

        #block J redblue
        self.J_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.J_resnet34[0] = nn.Conv2d(640, 64, kernel_size=7, stride=2, padding=3)
        self.J_resnet34 = nn.Sequential(
            self.J_resnet34,
            nn.Conv2d(512, 64, kernel_size=7, stride=2, padding=3)
        )
        self.J_block_convt = nn.ConvTranspose2d(in_channels=64,out_channels= 320, kernel_size=3, padding=1)
        self.J_block_bn = nn.BatchNorm2d(320)
        self.J_block_relu = nn.ReLU()

        #block K redblue
        self.K_in_block=nn.AdaptiveAvgPool2d((1,1))
        self.K_resnet34 = nn.Sequential(*list(self.resnet34.children())[:-2])
        self.K_resnet34[0] = nn.Conv2d(448, 64, kernel_size=7, stride=2, padding=3)
        self.K_resnet34 = nn.Sequential(
            self.K_resnet34,
            nn.Conv2d(512, 128, kernel_size=7, stride=2, padding=3)
        )
        self.K_block_convt = nn.ConvTranspose2d(in_channels=128,out_channels= 192, kernel_size=3, padding=1)
        self.K_block_bn = nn.BatchNorm2d(192)
        self.K_block_relu = nn.ReLU()

        #block L pinkblue
        self.L0_block_conv = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=34, padding=1)
        self.L1_block_conv = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.L1_block_bn = nn.BatchNorm2d(256)
        self.L1_block_relu = nn.ReLU()
        self.L2_block_convt = nn.ConvTranspose2d(in_channels=320, out_channels= 192, kernel_size=3, padding=1)
        self.L2_block_bn = nn.BatchNorm2d(192)
        self.L2_block_relu = nn.ReLU()

        #block M blue
        self.M_block_convt = nn.ConvTranspose2d(in_channels=192, out_channels=4, kernel_size=3, padding=1)
        self.M_block_bn = nn.BatchNorm2d(4)
        self.M_block_relu = nn.ReLU()
        self.M_avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        self.classifier = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(4, 2),  # Since your last layer outputs a tensor with 4 channels
            nn.Sigmoid()
        )
        self.bbox_regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4, 4)  
        )
        

    def forward(self, x, y):

        outA = self.A_block_relu(self.A_block_bn(self.A_block_conv(x)))
        #print("outA", outA.size())
        outC = self.C_resnet34(outA)
        #print("outC", outC.size())
        outE = self.E_resnet34(outC)
        #print("outE", outE.size())

        outB = self.B_block_relu(self.B_block_bn(self.B_block_conv(y)))
        #print("outB", outB.size())
        outD = self.D_resnet34(outB)
        #print("outD", outD.size())
        outF = self.F_resnet34(outD)
        #print("outF", outF.size())

        outG = self.G2_resnet34(self.G1_resnet34(torch.cat([outE, outF], dim=1)))
        #print("outG", outG.size())

        outH = self.H_block_relu(self.H_block_bn(self.H_block_conv(outG)))
        #print("outH", outH.size())

        outI = self.I2_block_relu(self.I2_block_bn(self.I2_block_conv(torch.cat([self.I1_block_relu(self.I1_block_bn(self.I1_block_conv(outH))), outH], dim=1))))
        #print("outI", outI.size())

        outJ = self.J_block_relu(self.J_block_bn(self.J_block_convt(self.J_resnet34(torch.cat([outI, outE, outF], dim=1)))))

        outC_pooled = self.K_in_block(outC)
        outD_pooled = self.K_in_block(outD)
        outK = self.K_block_relu(self.K_block_bn(self.K_block_convt(self.K_resnet34(torch.cat([outJ, outC_pooled, outD_pooled], dim=1)))))
        #print("outK", outK.size())

        outA2 = self.L0_block_conv(outA)
        outB2 = self.L0_block_conv(outB)
        #print("outB2", outB2.size())
        #print("outK", outK.size())
        outK_upsampled =F.interpolate(outK, size=(193, 193), mode='bilinear', align_corners=False)
        outL = self.L2_block_relu(self.L2_block_bn(self.L2_block_convt(torch.cat([self.L1_block_relu(self.L1_block_bn(self.L1_block_bn(torch.cat([outK_upsampled, outB2], dim=1)))), outA2], dim=1))))
        #print("outL", outL.size())

        outM = self.M_avg_pool(self.M_block_relu(self.M_block_bn(self.M_block_convt(outL))))
        #print("outM", outM.size())

        outN= self.classifier(outM)
        # print("outL", outL.size())
        
        bbox_output = self.bbox_regressor(outM)
        return outN,bbox_output

model = MyModel()

#print(model)

#summary(model, input_size=((32, 3, 128, 256), (32, 3, 128, 256)))

In [ ]:
logdir = "logs"
experiment = 'learning'

# Directory where logs will be saved. 
log_dir = os.path.join(logdir, experiment)

# initiate tensorboard summary writer
tb_writer = SummaryWriter(
    log_dir = log_dir,
    comment = "EarlyFusion"
)
train_accuracy_tag = 'accuracy/train'
validation_accuracy_tag = 'accuracy/validation'
train_loss_tag = 'loss/train'
validation_loss_tag = 'loss/validation'
training_ROC_tag='ROC/train'
validation_ROC_tag='ROC/validation'
training_pearson_tag='pearson/training'
validation_pearson_tag='pearson/validation'
training_kendal_tag='kendal/training'
validation_kendal_tag='kendal/validation'

In [ ]:
def save_model(model, device,epoch, model_dir='models', ):
    model_file_name=f'earlyfusion{epoch}.pth'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_path = os.path.join(model_dir, model_file_name)


    if device == 'cuda':
        model.to('cpu')

    # save the state_dict
    torch.save(model.state_dict(), model_path)
    # transfer the model to gpu.
    if device == 'cuda':
        model.to('cuda')

    return

In [ ]:
Mean= [0.485, 0.456, 0.406]
Std= [0.229, 0.224, 0.225]
common_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(Mean, Std)
    ])

trainset = kitti_loader(directory=data_root,transform=common_transforms)
valset = kitti_loader(directory=validation_root,transform=common_transforms)

batch_size = 2
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,drop_last=True)
valloader= torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True,drop_last=True)


optimizer = optim.AdamW(
    model.parameters(),
    lr = 0.0001)

num_epochs = 100
device = torch.device("cuda")
model = model.to(device)

kendall = KendallRankCorrCoef(num_outputs=4).cuda()
pearson = PearsonCorrCoef(num_outputs=4).cuda()

t_begin = time.time()
criterion = nn.MSELoss()
for epoch in range(num_epochs):
    running_loss = 0.0
    batch_kendall = []
    batch_pearson = []
    for i, data in enumerate(trainloader):
        input1, input2,label = data

        input1 = input1.cuda()
        input2 = input2.cuda()
        labels = label.cuda().squeeze()
        optimizer.zero_grad()
        outputs_class,outputs_bbox  = model(input1, input2)

        loss = criterion(outputs_bbox, labels.float())
        loss.backward()
        optimizer.step()
        if loss.item!=None:
            running_loss += loss.item()


        batch_kendall.append(kendall (outputs_bbox, labels.float()))
        batch_pearson.append(pearson(outputs_bbox, labels.float()))

    print(batch_kendall)
    kendall_sum = torch.stack(batch_kendall).sum(dim=0)  # sum size of batch
    kendall_mean = kendall_sum / len(trainloader)  # devide len batch
    kendall_mean_scalar = torch.mean(kendall_mean).item()  
    print("kendall_mean_scalar",kendall_mean_scalar)
    pearson_sum = torch.stack(batch_pearson).sum(dim=0)
    pearson_mean = pearson_sum / len(trainloader)
    pearson_mean_scalar = torch.mean(pearson_mean).item()
    print("pearson_mean_scalar",pearson_mean_scalar)
    tb_writer.add_scalar(tag=train_loss_tag, 
             scalar_value=running_loss,
             global_step=epoch)
    
    tb_writer.add_scalar(tag=training_kendal_tag, 
             scalar_value=kendall_mean_scalar,
             global_step=epoch)
    tb_writer.add_scalar(tag=training_pearson_tag, 
             scalar_value=pearson_mean_scalar,
             global_step=epoch)

    elapsed_time = time.time() - t_begin
    speed_epoch = elapsed_time / (epoch + 1)
    eta = speed_epoch * num_epochs - elapsed_time

    print(
        "Elapsed {:.2f}s, {:.2f} s/epoch, ets {:.2f}s".format(
            elapsed_time, speed_epoch, eta
        )
    )


    print(f"Epoch {epoch+1},Loss: {running_loss} ")

    if epoch % 5 == 0 :
        model.eval()
        val_loss = 0
        
        pearson_val = PearsonCorrCoef(num_outputs=4).cuda()
        kendall_val = KendallRankCorrCoef(num_outputs=4).cuda()
        batch_kendall_val = []
        batch_pearson_val = []
        for batch_idx, (data) in enumerate(valloader):
            input1, input2,label = data
            input1 = input1.cuda()
            input2 = input2.cuda()
            labels = label.cuda().squeeze()
    
            with torch.no_grad():
                outputs_class,outputs_bbox  = model(input1, input2)

            batch_kendall_val.append(kendall (outputs_bbox, labels.float()))
            batch_pearson_val.append(pearson(outputs_bbox, labels.float()))
        
        kendall_sum_val = torch.stack(batch_kendall_val).sum(dim=0)
        kendall_mean_val = kendall_sum_val / len(valloader)
        kendall_mean_val_scalar = torch.mean(kendall_mean_val).item()
        
        pearson_sum_val = torch.stack(batch_pearson_val).sum(dim=0)
        pearson_mean_val= pearson_sum_val/ len(valloader)
        pearson_mean_val_scalar = torch.mean(pearson_sum_val).item()

        tb_writer.add_scalar(tag=validation_loss_tag, 
             scalar_value=val_loss,
             global_step=epoch)
        tb_writer.add_scalar(tag=validation_kendal_tag, 
                 scalar_value=kendall_mean_val_scalar,
                 global_step=epoch)
        tb_writer.add_scalar(tag=validation_pearson_tag, 
                 scalar_value=pearson_mean_val_scalar,
                 global_step=epoch)

        save_model(model, device=device,epoch=epoch)



print("Finished Training")